#### Создадим базу данных с именем "fanuc_primary_base"

In [26]:
import psycopg2

conn = psycopg2.connect(dbname="postgres", user="postgres", password="nbvjirF9291", host="localhost", port="5432")
cursor = conn.cursor()

conn.autocommit = True
# команда для создания базы данных fanuc_primary_base
sql = "CREATE DATABASE fanuc_primary_base"

# выполняем код sql
cursor.execute(sql)
print("База данных успешно создана")

cursor.close()
conn.close()

База данных успешно создана


#### В базе данных "fanuc_primary_base" создадим таблицу primary_machine_data

In [28]:
import psycopg2

conn = psycopg2.connect(dbname="fanuc_primary_base", user="postgres", password="nbvjirF9291", host="localhost", port="5432")
cursor = conn.cursor()

# создаем таблицу primary_machine_data
cursor.execute("""
    CREATE TABLE IF NOT EXISTS primary_machine_data (
        Logno bigserial,
        machinename VARCHAR(25),
        statustype smallint,
        stdatetime timestamp,
        enddatetime timestamp,
        partcount smallint,
        progno VARCHAR(8),
        cyclerecord BOOL,
        iscontinuous BOOL,
        continuouslogno smallint,
        transactiontime timestamp,
        transactionms smallint,
        almmsg VARCHAR(255),
        controlpartcount smallint
    );
""")

# поддверждаем транзакцию
conn.commit()
print("Таблица people успешно создана")

cursor.close()
conn.close()

Таблица people успешно создана


#### Добавляем все данные в базу SQL

In [39]:
#импортировать необходимые библиотеки
import pyodbc
import psycopg2

# Подключитесь к базе данных Access
access_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    # r'DBQ=E:\MONITORING\test_pandas.accdb;'
    r'DBQ=D:\SkyMarsUtilization\2022-02-09_UtilizationDB.mdb'
    )
access_conn = pyodbc.connect(access_conn_str)
access_cursor = access_conn.cursor()

# Подключитесь к базе данных PostgreSQL
pg_conn = psycopg2.connect(dbname="fanuc_primary_base", user="postgres", password="nbvjirF9291", host="localhost", port="5432")
pg_cursor = pg_conn.cursor()

# Выбор новых данных из базы данных Access
access_cursor.execute("SELECT * FROM StatusLog")
new_data = set([tuple(row) for row in access_cursor.fetchall()])

# Вставка новых данных в базу данных PostgreSQL
pg_cursor.executemany("INSERT INTO primary_machine_data VALUES (%s, %s, %s, %s,\
       %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", new_data)

# Зафиксируйте изменения в базе данных PostgreSQL
pg_conn.commit()
print("Данные успешно добавленны")

# Close connections
pg_cursor.close()
pg_conn.close()
access_cursor.close()
access_conn.close()

#### Добавляем в базу SQL только новые данные

In [1]:
# Импорт необходимых библиотек
import pyodbc
import psycopg2

# Подключение к базе данных Access
access_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=D:\SkyMarsUtilization\2022-02-09_UtilizationDB.mdb'
)
with pyodbc.connect(access_conn_str) as access_conn, access_conn.cursor() as access_cursor:

    # Подключение к базе данных PostgreSQL
    pg_conn = psycopg2.connect(
        dbname="fanuc_primary_base",
        user="postgres",
        password="nbvjirF9291",
        host="localhost",
        port="5432"
    )
    with pg_conn, pg_conn.cursor() as pg_cursor:

        # Получение существующих данных из базы данных PostgreSQL
        pg_cursor.execute(
            "SELECT logno, machinename, statustype, stdatetime, enddatetime, partcount, progno, cyclerecord, iscontinuous, \
            continuouslogno, transactiontime, transactionms, almmsg, controlpartcount FROM primary_machine_data"
        )
        existing_data = {row[0]: row[1:] for row in pg_cursor.fetchall()}

        # Получение новых данных из базы данных Access
        access_cursor.execute("SELECT * FROM StatusLog")
        new_data = [tuple(row) for row in access_cursor.fetchall()]

        # Отфильтруйте существующие данные из new_data
        new_data = [row for row in new_data if row[0] not in existing_data.keys()]

        # Вставка новых_данных в базу данных PostgreSQL
        pg_cursor.executemany(
            "INSERT INTO primary_machine_data (logno, machinename, statustype, stdatetime, enddatetime, partcount, progno, \
            cyclerecord, iscontinuous, continuouslogno, transactiontime, transactionms, almmsg, controlpartcount) \
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
            new_data
        )

        # Зафиксируйте изменения в базе данных PostgreSQL
        pg_conn.commit()

In [ ]:
# Write python code that will connect to the CNC machine (ip 192.168.105.9) using the Fanuc Focas library located in folder D:\FANUC FOCAS\FANUC FOCAS and read from it the spindle load, number of parts and current program number

import os
import sys
from ctypes import *
from time import sleep
from pathlib import Path

# Define the path to the Focas library
dll_path = Path("D:/FANUC FOCAS/FANUC FOCAS/Fwlib64/Fwlib64.dll")

# Check if the Focas library exists
if not dll_path.exists():
    print("Focas library not found!")
    sys.exit(1)

# Open the Focas library
try:
    with dll_path.open() as f:
        pass
except PermissionError:
    print("Permission denied to open Focas library!")
    sys.exit(1)

# Load the Focas library
fwlib = cdll.LoadLibrary(str(dll_path))

# Connect to the CNC machine
handle = c_long(0)
ip_address = b"192.168.105.9"
port = 8193
fwlib.cnc_allclibhndl3(ip_address, port, 10, byref(handle))

# Read spindle load
load = c_float()
fwlib.cnc_rdspload(handle, byref(load))
print("Spindle Load:", load.value)

# Read number of parts
num_parts = c_long()
fwlib.cnc_rdprgnum(handle, byref(num_parts))
print("Number of Parts:", num_parts.value)

# Read current program number
prog_num = c_long()
fwlib.cnc_rdprgnum(handle, byref(prog_num))
print("Current Program Number:", prog_num.value)

# # Read current feed rate
# feed_rate = c_float()
# fwlib.cnc_rdfdrate(handle, byref(feed_rate))
# print("Current Feed Rate:", feed_rate.value)

# Read current spindle speed
spindle_speed = c_long()
fwlib.cnc_rdparainfo(handle, byref(spindle_speed))
print("Current Spindle Speed:", spindle_speed.value)

# # Read part counter
# part_counter = c_long()
# fwlib.cnc_rdprcnt(handle, byref(part_counter))
# print("Part Counter:", part_counter.value)

# Release the handle
fwlib.cnc_freelibhndl(handle)


In [ ]:
# Write instructions on how to connect to a CNC machine with ip address 192.168.105.102 using the repository https://github.com/strangesast/fwlib/tree/docker-ci-improvements on a windows 10 system



Connecting to CNC Machine with IP Address 192.168.105.102 using fwlib on Windows 10:

Pseudocode:

1. Install Git for Windows by downloading it from https://git-scm.com/downloads and following the installation instructions.
2. Open Git Bash and navigate to the directory where you want to clone the fwlib repository.
3. Clone the fwlib repository by running the following command: git clone https://github.com/strangesast/fwlib.git
4. Navigate to the "/fwlib/docker-ci-improvements" directory by running the following command: cd fwlib/docker-ci-improvements
5. Build the Docker container by running the following command: docker build --tag fwlib-cnc .
6. Start the Docker container and map port 8193 to the CNC machine's IP address by running the following command: docker run -p 8193:8193 -e CNC_HOST=192.168.105.102 fwlib-cnc
7. You can now connect to the CNC machine by opening a web browser and navigating to http://localhost:8193/

Code:

```python
# Step 1: Install Git for Windows
# Download from: https://git-scm.com/downloads
# Follow installation instructions

# Step 2: Open Git Bash and navigate to the directory where you want to clone the fwlib repository
# Assuming the directory is "C:\Users\John\Documents"
cd C:/Users/John/Documents

# Step 3: Clone the fwlib repository
git clone https://github.com/strangesast/fwlib.git

# Step 4: Navigate to the "/fwlib/docker-ci-improvements" directory
cd fwlib/docker-ci-improvements

# Step 5: Build the Docker container
docker build --tag fwlib-cnc .

# Step 6: Start the Docker container and map port 8193 to the CNC machine's IP address
docker run -p 8193:8193 -e CNC_HOST=192.168.105.102 fwlib-cnc

# Step 7: Connect to the CNC machine by opening a web browser and navigating to http://localhost:8193/
```